In [7]:
import os
from dotenv import load_dotenv
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAI
from qdrant_client import QdrantClient, models
import qdrant_client
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
# Set the model name for LLM
OPENAI_MODEL = "gpt-3.5-turbo"

# Store API key as a variable
openai_api_key = st.secrets["OPENAI_API_KEY"]

In [3]:
def get_vector_store():
    #Create a client to connect to Qdrant server
    client = qdrant_client.QdrantClient(
        st.secrets["QDRANT_HOST"],
        api_key=st.secrets["QDRANT_API_KEY"]
        )
    
    #initialize embeddings for vector store
    embeddings = OpenAIEmbeddings()
    
    # create vector_store with Qdrant and embeddings
    vector_store = Qdrant(client,
                          collection_name = st.secrets["QDRANT_COLLECTION_NAME"],
                          embeddings=embeddings)
    return vector_store

In [8]:
vector_store = get_vector_store()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\mered\\anaconda3\\Library\\ssl\\cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\mered\\anaconda3\\Library\\ssl\\cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\mered\\anaconda3\\Library\\ssl\\cacert.pem'


In [9]:
#initialize conversational retrieval chain
def initialize_crc(vector_store):
    llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0, api_key=openai_api_key)
    retriever=vector_store.as_retriever()
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    # st.session_state['crc'] = crc
    # st.success('Source documents loaded!')
    return crc 

In [10]:
vector_store = get_vector_store()
crc = initialize_crc(vector_store)
print(crc)  # Inspect the crc object

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\mered\\anaconda3\\Library\\ssl\\cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\mered\\anaconda3\\Library\\ssl\\cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\mered\\anaconda3\\Library\\ssl\\cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\mered\\anaconda3\\Library\\ssl\\cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Users\\mered\\anaconda3\\Library\\ssl\\cacert.pem'


combine_docs_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C3DE1A55D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C3DE1A6DD0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')), document_variable_name='context') question_generator=LLMChain(prompt=PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephr

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model=OPENAI_MODEL, temperature=0, api_key=openai_api_key),
    # chain_type="stuff",
    retriever = vector_store.as_retriever(metadata_fields=['metadata'])
    
)

In [12]:
query = "How does ASC 606 interact with ASC 842?"
chat_history = []
response = crc.run({
    'question': query,
    'chat_history': chat_history
})

print(response)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x000001C3DD0667A0>, 'json_data': {'input': [[4438, 1587, 20382, 220, 20213, 16681, 449, 20382, 220, 25377, 30]], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001C3DD075D50>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000001C3DE14F410> server_hostname='api.openai.com' timeout=None
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001C3DF145310>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DE

ASC 606 (Revenue from Contracts with Customers) and ASC 842 (Leases) interact in several ways. ASC 842 aligns certain concepts with ASC 606, such as determining whether the transfer of an asset is a sale in a sale and leaseback transaction. Additionally, ASC 842 requires lessors to apply specific guidance in ASC 606 to their leasing transactions, particularly in the allocation of consideration in the contract. The FASB clarified that entities do not need to revisit the allocation of contract consideration to lease components for existing unmodified contracts upon the adoption of ASC 842, except when lease classification changes. Furthermore, under ASC 842, certain maintenance activities that were considered executory costs under ASC 840 are now treated as non-lease components and accounted for under ASC 606 by lessors.


In [4]:
def get_vector_store():
    # Create a client to connect to Qdrant server
    client = qdrant_client.QdrantClient(
        st.secrets["QDRANT_HOST"],
        api_key=st.secrets["QDRANT_API_KEY"]
    )

    # Initialize embeddings for vector store
    embeddings = OpenAIEmbeddings()

    # Create a vector store with Qdrant and embeddings
    vector_store = Qdrant(client,
                          collection_name=st.secrets["QDRANT_COLLECTION_NAME"],
                          embeddings=embeddings)
    
    # Get the number of vectors in the Qdrant collection
    collection_info = client.get_collection(collection_name=st.secrets["QDRANT_COLLECTION_NAME"])
    vector_count = collection_info.points_count

    print(f"Vector store contains {vector_count} vectors.")

    return vector_store
    